In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install transformers datasets

In [3]:
from datasets import Dataset, load_dataset, load_metric
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
import torch
import pandas as pd
import json
from tqdm import tqdm

In [4]:
task = "ner"

label_list = ['B-PREDICATE-ACT', 'B-ARGUMENT-PLACE',
              'B-ARGUMENT-PATIENT', 'B-ARGUMENT-AGENT',
              'B-PREDICATE-STATE', 'I-PREDICATE-ACT',
              'I-ARGUMENT-PLACE', 'I-ARGUMENT-AGENT',
              'I-ARGUMENT-PATIENT', 'I-PREDICATE-STATE',
              'O']
label_encoding_dict = {value: idx for idx, value in enumerate(label_list)}

In [6]:
def get_token_dataset(sentences):
    list_tokens, list_labels = [], []
    for sentence in sentences:
        list_tokens.append(sentence.split())
        list_labels.append(["O"] * len(list_tokens[-1]))
    return Dataset.from_pandas(pd.DataFrame({'tokens': list_tokens, 'ner_tags': list_labels}))

In [5]:
def tokenize_and_align_labels(examples):
    global tokenizer
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)
    all_word_ids = []
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        all_word_ids.append(word_ids)
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = all_word_ids
    return tokenized_inputs

In [7]:
def predict(sentence):
    list_predictions = []
    test_dataset = get_token_dataset(sentence)
    # test_dataset = get_token_dataset([df["contexts"][data_i] for data_i in range(len(df))])
    test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

    trainer = Trainer(model, eval_dataset=test_tokenized_datasets, data_collator=data_collator)
    predictions, pseudo_labels, _ = trainer.predict(test_tokenized_datasets)
    predictions = np.argmax(predictions, axis=2)

    list_prediction_probabily = []
    word_ids = test_tokenized_datasets["word_ids"]
    
    tokens = test_dataset['tokens']
    for idx, (prediction, pseudo_label) in tqdm(enumerate(zip(predictions, pseudo_labels))):
        prediction_probabily = []
        for i, (p, l) in enumerate(zip(prediction, pseudo_label)):
            if l == -100: continue
            if word_ids[idx][i] == word_ids[idx][i-1]: 
                continue
            prediction_probabily.append(label_list[p])
       
        if len(prediction_probabily) != len(tokens[idx]):
            print(len(prediction_probabily), len(tokens[idx]))
            print((prediction_probabily), (tokens[idx]))
            error_files.add(idx)
            for k in range(len(prediction_probabily), len(tokens[idx])):
                prediction_probabily.append("O")
            print(len(prediction_probabily), len(tokens[idx]))

        list_predictions.append(prediction_probabily)
    return list_predictions

In [8]:
def merge(all_sentence, num_sentences_in_para): # Merge sentence into paragraph using num_sentences_in_para

    paragraphs = [] 
    previous_num_sentences = 0
    for num_sentences in num_sentences_in_para:
        para = []
        for sentence in all_sentence[previous_num_sentences:previous_num_sentences+num_sentences]:
            para.append(sentence)
        paragraphs.append(para)
        previous_num_sentences += num_sentences
    return paragraphs

In [9]:
def start_end_index(example):
    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for i, token in enumerate(example['contexts'].split()):
        for t in range(len(token)+1):
            tok_to_orig_index.append(i)
        all_doc_tokens.append(token)
    if example['is_impossible']:
        start_index = -1
        end_index = -1
    else:
        start_index = tok_to_orig_index[int(example['start'])]
        end_index = tok_to_orig_index[int(example['end'])]
    return start_index, end_index

In [ ]:
df['answer_text'] = df_org['answers']
start_end = [start_end_index(df_org.iloc[i]) for i in range(len(df))]
df['start'] = [se[0] for se in start_end]
df['end'] = [se[-1] for se in start_end]

NameError: ignored

In [ ]:
df = pd.read_csv(f"/content/drive/MyDrive/KLTN/srl/data_original/train_5.csv")

In [ ]:
df.head()

,id,contexts,questions,is_impossible,answers,start,end
0,uit_000001,Phạm Văn Đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,False,Lâm Bá Kiệt,522.0,533.0
1,uit_000002,Phạm Văn Đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,False,Thủ tướng,62.0,71.0
2,uit_000003,Phạm Văn Đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,"Giai đoạn năm 1955 - 1976 , Phạm Văn Đồng nắm ...",False,Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa,250.0,295.0
3,uit_000004,Phạm Văn Đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,True,NaN,NaN,NaN
4,uit_000005,Phạm Văn Đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,True,NaN,NaN,NaN


In [10]:
files = ["train_5", "dev_5"]
for n_cp in range(10):
    model_checkpoint = f'/content/drive/MyDrive/KLTN/srl/SRL_final/xlm-roberta-large-finetuned-ner-{n_cp+1}/checkpoint-15000'
    for f in files:
        print("model_checkpoint: ",model_checkpoint,"\nfile: ", f)
        df = pd.read_csv(f"/content/drive/MyDrive/KLTN/srl/data_original/{f}.csv")
        print("model_checkpoint: ", model_checkpoint)
        df.columns = ["id",'contexts', 'questions', "labels", "answers","start","end"]
        df['sample_id'] = range(1, len(df) + 1)
        df = df[['sample_id',"id",'contexts', 'questions', "labels", "answers","start","end"]]

        sample_ids = [df["sample_id"][i] for i in range(len(df))]

        error_files = set()
        list_predictions = []
        
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
        data_collator = DataCollatorForTokenClassification(tokenizer)

        # Split all paragraphs to sentences and store in all_sentences 
        all_sentence = []
        num_sentences_in_para = [] # the list of the number of sentences in paragraphs
        for paragraph in df['contexts']:
            num_sentences_in_para.append(len(paragraph.split(" . ")))
            for sentence in paragraph.split(" . "):
                all_sentence.append(sentence)
        
        prediction_lst = []
        model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_encoding_dict))
        print("predicting...")
        print(len(all_sentence))
        predict_sentence = predict(all_sentence)

        
        print("Predict sentence completely!")
        question_pred_list = predict(df['questions'])
        print("Predict question completely!")

        lst_tag_predict = merge(predict_sentence, num_sentences_in_para)
        s_list = []
        for tag_para in lst_tag_predict:
            s = []
            for tag_sen in tag_para:
                for tag in tag_sen:
                    s.append(tag)
                s.append('O')
            s_list.append(s[:-1])
        df_tags = pd.DataFrame({"contexts":df['contexts'], 
                        "context_tags":s_list, 
                        "questions":df['questions'],
                        "question_tags":question_pred_list,
                        "labels":df['labels'] })
        df_tags.to_csv(f"/content/drive/MyDrive/KLTN/srl/data/data_prediction_{f}_{n_cp+1}.csv", index=False)

model_checkpoint:  /content/drive/MyDrive/KLTN/srl/SRL_final/xlm-roberta-large-finetuned-ner-1/checkpoint-15000 
file:  train_5
model_checkpoint:  /content/drive/MyDrive/KLTN/srl/SRL_final/xlm-roberta-large-finetuned-ner-1/checkpoint-15000
predicting...
117502


Parameter 'function'=<function tokenize_and_align_labels at 0x7fa068d3aef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/118 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: word_ids, tokens, ner_tags. If word_ids, tokens, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 117502
  Batch size = 8


65069it [00:10, 6288.81it/s]

...

Predict sentence completely!


  0%|          | 0/8 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: word_ids, tokens, ner_tags. If word_ids, tokens, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7223
  Batch size = 8


7223it [00:00, 45392.44it/s]


Predict question completely!


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KLTN/srl/data_original/dev_5.csv")
df.columns = ['contexts', 'questions', "labels"]
df['sample_id'] = range(1, len(df) + 1)
df = df[['sample_id', 'contexts', "questions","labels"]]

In [ ]:
df.head(5)

,sample_id,contexts,questions,labels
0,1,"Nhôm ( bắt nguồn từ tiếng Pháp : aluminium , p...",Nhôm nằm ở ô số mấy trong bảng tuần hoàn hóa h...,False
1,2,"Nhôm ( bắt nguồn từ tiếng Pháp : aluminium , p...","Trong vỏ trái đất , nguyên tố nào chiếm thành ...",False
2,3,"Nhôm ( bắt nguồn từ tiếng Pháp : aluminium , p...",Nhiệt độ cực tiểu để làm nhôm chuyển từ thể rắ...,False
3,4,"Nhôm ( bắt nguồn từ tiếng Pháp : aluminium , p...",Nhôm thường tồn tại ở dạng nào trong lòng đất ?,False
4,5,"Nhôm ( bắt nguồn từ tiếng Pháp : aluminium , p...",Nguyên liệu chính dùng để sản xuất nhôm là gì ?,False


In [ ]:
sample_ids = [df["sample_id"][i] for i in range(len(df))]

error_files = set()
list_predictions = []

In [ ]:
# Split all paragraphs to sentences and store in all_sentences 
all_sentence = []
num_sentences_in_para = [] # the list of the number of sentences in paragraphs
for paragraph in df['contexts']:
    num_sentences_in_para.append(len(paragraph.split(" . ")))
    for sentence in paragraph.split(" . "):
        all_sentence.append(sentence)

In [ ]:
prediction_lst = []
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_encoding_dict))
predict_sentence = predict(all_sentence)

All model checkpoint weights were used when initializing RemBertForTokenClassification.

All the weights of RemBertForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/KLTN/srl/SRL_train_4/rembert-finetuned-ner-1/checkpoint-15000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RemBertForTokenClassification for predictions without further training.


  0%|          | 0/15 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `RemBertForTokenClassification.forward` and have been ignored: tokens, word_ids, ner_tags. If tokens, word_ids, ner_tags are not expected by `RemBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 14202
  Batch size = 8


4754it [00:00, 11464.11it/s]

170 171
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['Các', 'món', 'xôi', 'thường', 'thấy', 'là', 'xôi', 'vò', '(', 'xôi', 'trộn', 'đậu', 'xanh', 'giã', 'mịn', ',', 'làm', 'tơi', 'từng', 'hạt',

14202it [00:01, 11917.18it/s]


In [ ]:
lst_tag_predict = merge(predict_sentence, num_sentences_in_para)

In [ ]:
s_list = []
for tag_para in lst_tag_predict:
    s = []
    for tag_sen in tag_para:
        for tag in tag_sen:
            s.append(tag)
        s.append('O')
    s_list.append(s[:-1])

In [ ]:
question_pred_list = predict(df['questions'])

  0%|          | 0/3 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `RemBertForTokenClassification.forward` and have been ignored: tokens, word_ids, ner_tags. If tokens, word_ids, ner_tags are not expected by `RemBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2717
  Batch size = 8


2717it [00:00, 45354.20it/s]


In [ ]:
df_tags = pd.DataFrame({"contexts":df['contexts'], 
                        "context_tags":s_list, 
                        "questions":df['questions'],
                        "question_tags":question_pred_list,
                        "labels":df['labels'] })
df_tags.to_csv("/content/drive/MyDrive/KLTN/srl/data/data_prediction_10.csv", index=False)

In [ ]:
/content/drive/MyDrive/KLTN/srl/data/ 